Загрузим данные

In [21]:
import pandas as pd
import random
import pickle
import torch
random.seed(123123)

In [22]:
df_test = pd.read_csv("data/polyphar_test.csv")
df_train = pd.read_csv("data/polyphar_train.csv")
df_val = pd.read_csv("data/polyphar_validate.csv")

In [23]:
print(df_test.head())
print(df_val.head())
print(pd_train.head())

print(len(df_test))
print(len(df_train))
print(len(df_val))

   from  rel   to
0   624  442  350
1   144  442  118
2   478  442   70
3    93  442  441
4   238  442  266
   from  rel   to
0   451  442  234
1   512  442  157
2   570  442  350
3   375  442  299
4   417  442  299
   from  rel   to
0   243  442  441
1   238  442  526
2   451  442  299
3   230  442  526
4   297  442  622
457196
3661895
457196


Посмотим, где что есть.

In [24]:
train_drugs = set(df_train["from"]).union(set(pd_train["to"]))
test_drugs = set(pd_test["from"]).union(set(df_test["to"]))
val_drugs = set(df_val["from"]).union(set(df_val["to"]))
test_and_val_drugs = test_drugs.union(val_drugs)

In [25]:
print(len(train_drugs.intersection(test_and_val_drugs)))
print(len(train_drugs))
print(len(test_and_val_drugs))
print(len(val_drugs))
print(len(test_drugs))

645
645
645
645
644


In [26]:
train_from = set(df_train["from"])
train_to = set(df_train["to"])

print(len(train_from))
print(len(train_to))

616
634


Выберем случайно (seed уже зафиксирован) 50 лекарств, которые уберем из train целиком. И по 20, для которых оставим только односторонние связи. Сохраним все.

In [35]:
group_all = random.sample(train_drugs, 50)
group_from = random.sample(train_from.difference(group_all), 20)
group_to = random.sample(train_to.difference(group_all).difference(group_from), 20)

with open("data/removed_drugs", "wb") as f:
    pickle.dump((group_all, group_from, group_to), f)

df_train = df_train[~df_train["from"].isin(group_all)]
df_train = df_train[~df_train["to"].isin(group_all)]
df_train = df_train[~df_train["from"].isin(group_from)]
df_train = df_train[~df_train["to"].isin(group_to)]

df_train.to_csv("data/polyphar_train_modified.csv",index=False)


<ipython-input-35-ea7e9a8a678e>:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  group_all = random.sample(train_drugs, 50)
<ipython-input-35-ea7e9a8a678e>:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  group_from = random.sample(train_from.difference(group_all), 20)
<ipython-input-35-ea7e9a8a678e>:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  group_to = random.sample(train_to.difference(group_all).difference(group_from), 20)


Обучим на 8 эпохах и сравним метрики, полученные во время обучения.

test:

|          | test_all_auc_roc | test_all_auprc | test_all_ap50 |
|----------|------------------|----------------|---------------|
| Clean    | 0.967            | 0.94           | 0.977         |
| Modified | 0.67             | 0.74           | 0.93          |

Графики с neptune на train сильно не отличаются, итоговые значения val практически такие же, как и у test, а во время обучения они почти не менялись.